# Final Project
ChihHao Luca Yuan

In [31]:
import nltk
from nltk.corpus import sentence_polarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import random
import pandas as pd
import numpy as np
import re
import os
import sys
from sklearn.model_selection import train_test_split

In [32]:
train = pd.read_csv("train.tsv", sep='\t', index_col="PhraseId")

In [33]:
test = pd.read_csv("test.tsv", sep='\t', index_col="PhraseId")

In [34]:
submission = pd.read_csv("sampleSubmission.csv")

In [35]:
train.head(10)

,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2
4,1,A,2
5,1,series,2
6,1,of escapades demonstrating the adage that what...,2
7,1,of,2
8,1,escapades demonstrating the adage that what is...,2
9,1,escapades,2


In [36]:
phrasedata = train[['Phrase', 'Sentiment']]

In [38]:
phrasedata

,Phrase,Sentiment
PhraseId,,
1,A series of escapades demonstrating the adage ...,1
2,A series of escapades demonstrating the adage ...,2
3,A series,2
4,A,2
5,series,2
...,...,...
156056,Hearst 's,2
156057,forced avuncular chortles,1
156058,avuncular chortles,3


In [39]:
print(type(phrasedata))

<class 'pandas.core.frame.DataFrame'>


In [40]:
phrasedata = phrasedata.values.tolist()


In [41]:
print(type(phrasedata))

<class 'list'>


In [42]:
limit = int(20000)
random.shuffle(phrasedata)
phraselist = phrasedata[:limit]


In [43]:
print('Read', len(phrasedata), 'phrases, using', len(phraselist), 'random phrases')

Read 156060 phrases, using 20000 random phrases


In [44]:
for phrase in phraselist[:10]:
    print (phrase)

['that movie', 2]
['a mediocre collection', 2]
['movie types', 2]
['Christmas !', 2]
['was produced by Jerry Bruckheimer and directed by Joel Schumacher , and reflects the worst of their shallow styles :', 1]
['especially charm', 3]
['mind images of a violent battlefield action picture', 2]
["The movie 's gloomy atmosphere is fascinating , though , even if the movie itself does n't stand a ghost of a chance .", 2]
['a window', 2]
['has less spice', 2]


In [45]:
  # create list of phrase documents as (list of words, label)
  phrasedocs = []
  # add all the phrases
  for phrase in phraselist:
    tokens = nltk.word_tokenize(phrase[0])
    tokens = [token.lower() for token in tokens]
    phrasedocs.append((tokens, int(phrase[1])))

In [46]:
total_rows = len(phrasedocs)
total_rows

20000

In [47]:
all_words_list = [word for (sent,sentiment) in phrasedocs for word in sent]
all_words = nltk.FreqDist(all_words_list)

word_items = all_words.most_common(2000)
word_features = [word for (word, freq) in word_items]
print(word_features[:50])

['the', ',', 'a', 'of', 'and', 'to', '.', "'s", 'in', 'is', 'that', 'it', 'as', 'with', 'for', 'its', 'an', 'film', 'movie', 'this', 'but', 'on', 'you', 'be', 'more', 'his', 'by', '``', "n't", '--', 'one', 'from', 'about', 'at', 'than', 'not', 'or', 'have', 'like', 'all', 'are', "'", 'has', '-rrb-', 'story', 'so', 'out', 'who', 'into', '-lrb-']


In [48]:
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    return features

In [49]:
featuresets = [(document_features(d,word_features), c) for (d,c) in phrasedocs]

In [50]:
from sklearn.model_selection import train_test_split

In [22]:
train_set, test_set = train_test_split(featuresets, test_size=0.2, random_state=42)   #42

## Accuracy : 0.56725 - 2000 most frequency words.

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(classifier, test_set))

0.56725


Cross Valid

In [64]:
label_list = [c for (d,c) in phrasedocs_2]
labels = list(set(label_list))    # gets only unique labels
num_folds = 3

In [ ]:
label_list = [c for (d,c) in phrasedocs]
labels = list(set(label_list))    # gets only unique labels
num_folds = 3
cross_validation_PRF(num_folds, featuresets, labels)

Each fold size: 6666
Fold 0
Fold 1
Fold 2

Average Precision	Recall		F1 	Per Label
0 	      0.243      0.210      0.225
1 	      0.277      0.418      0.333
2 	      0.827      0.637      0.720
3 	      0.260      0.429      0.324
4 	      0.229      0.315      0.264

Macro Average Precision	Recall		F1 	Over All Labels
	      0.367      0.402      0.373

Label Counts {0: 897, 1: 3503, 2: 10173, 3: 4223, 4: 1204}
Micro Average Precision	Recall		F1 	Over All Labels
	      0.549      0.516      0.519


## Remove Stop words -> Bag of Words

In [51]:
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing',
                 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

In [52]:
stopwords = nltk.corpus.stopwords.words('english')

In [53]:
len(stopwords)

179

In [54]:
newstopwords = [word for word in stopwords if word not in negationwords]

In [55]:
len(newstopwords)

176

In [56]:
newstopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [57]:
all_words_list_wo_stopwords = [word for word in all_words_list if word not in newstopwords]

In [58]:
new_all_words = nltk.FreqDist(all_words_list_wo_stopwords)

In [59]:
new_word_items = new_all_words.most_common(2000)

In [60]:
new_word_features = [word for (word, count) in new_word_items]

In [ ]:
featuresets_wo_stopwords = [(document_features(d,new_word_features), c) for (d,c) in phrasedocs]

In [ ]:
wo_stopwords_train_set, wo_stopwords_test_set = train_test_split(featuresets_wo_stopwords, test_size=0.2, random_state=42)   #42

## Accuracy : 0.575

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(wo_stopwords_train_set)
print (nltk.classify.accuracy(classifier, wo_stopwords_test_set))

0.575


In [45]:
cross_validation_PRF(num_folds, featuresets_wo_stopwords, labels)

Each fold size: 6666
Fold 0
Fold 1
Fold 2

Average Precision	Recall		F1 	Per Label
0 	      0.183      0.246      0.210
1 	      0.273      0.426      0.333
2 	      0.843      0.632      0.723
3 	      0.286      0.438      0.346
4 	      0.199      0.315      0.244

Macro Average Precision	Recall		F1 	Over All Labels
	      0.357      0.411      0.371

Label Counts {0: 897, 1: 3503, 2: 10173, 3: 4223, 4: 1204}
Micro Average Precision	Recall		F1 	Over All Labels
	      0.557      0.519      0.523


## lemma process -> stopwords -> BoW

In [61]:


phrasedocs_2 = []
for phrase in phraselist:
    tokens = nltk.word_tokenize(phrase[0])  
    tokens = [token.lower() for token in tokens]  
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    phrasedocs_2.append((lemmatized_tokens, int(phrase[1])))


In [36]:
featuresets_wo_stopwords_lemma = [(document_features(d,new_word_features), c) for (d,c) in phrasedocs_2]

In [37]:
wo_stopwords_train_set_lemma, wo_stopwords_test_set_lemma = train_test_split(featuresets_wo_stopwords_lemma, test_size=0.2, random_state=42)   #42

0.57575

In [61]:
classifier = nltk.NaiveBayesClassifier.train(wo_stopwords_train_set_lemma)
print (nltk.classify.accuracy(classifier, wo_stopwords_test_set_lemma))

0.57575


### Remove Stop Words then Negation words 

In [38]:
def NOT_features(document, word_features, negationwords):
    features = {}
    
    for word in word_features:
        features['contains({})'.format(word)] = False
        features['contains(NOT{})'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['contains(NOT{})'.format(document[i])] = (document[i] in word_features)
        else:
            features['contains({})'.format(word)] = (word in word_features)
    return features


In [41]:
NOT_featuresets = [(NOT_features(d, new_word_features, negationwords), c) for (d, c) in phrasedocs_2]


In [66]:
train_set_NOT_Feature, test_set_NOT_Feature = train_test_split(NOT_featuresets, test_size=0.2, random_state=42)

## Remove stop words + negation, Accuracy = 0.51525 (freq 2000 words)

In [67]:
classifier = nltk.NaiveBayesClassifier.train(train_set_NOT_Feature)
print (nltk.classify.accuracy(classifier, test_set_NOT_Feature))

0.51525


+ Subjectivity

In [42]:
def readSubjectivity(path):
	flexicon = open(path, 'r')
	# initialize an empty dictionary
	sldict = { }
	for line in flexicon:
		fields = line.split()   # default is to split on whitespace
		# split each field on the '=' and keep the second part as the value
		strength = fields[0].split("=")[1]
		word = fields[2].split("=")[1]
		posTag = fields[3].split("=")[1]
		stemmed = fields[4].split("=")[1]
		polarity = fields[5].split("=")[1]
		if (stemmed == 'y'):
			isStemmed = True
		else:
			isStemmed = False
		# put a dictionary entry with the word as the keyword
		#     and a list of the other values
		sldict[word] = [strength, posTag, isStemmed, polarity]
	return sldict

In [43]:
SLpath = "/Users/simba/Desktop/semester 3/NLP/Final Project/FinalProjectData (5)/FinalProjectData/kagglemoviereviews/corpus/SentimentLexicons/subjclueslen1-HLTEMNLP05.tff"

In [44]:
SL = readSubjectivity(SLpath)

In [45]:
def SL_features(document, SL):
    document_words = set(document)
    features = {}
    
    for word in new_word_features:
        features['contains(%s)' % word] = (word in document_words)
        
    # count variables for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    
    for word in document_words:
        if word in SL:
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'weaksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
                
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)
    
    return features


In [46]:
SL_featuresets = [(SL_features(d, SL), c) for (d,c) in phrasedocs_2]

## SL : 0.576

In [ ]:
train_set_SL, test_set_SL = train_test_split(SL_featuresets, test_size=0.2, random_state=42)   #42

classifier = nltk.NaiveBayesClassifier.train(train_set_SL)
print(nltk.classify.accuracy(classifier, test_set_SL))

0.576


In [ ]:
def SL_features(document, SL):
    document_words = set(document)
    features = {}
    
    for word in new_word_features:
        features['contains(%s)' % word] = (word in document_words)
        
    # count variables for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    
    for word in document_words:
        if word in SL:
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'weaksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
                
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)
    
    return features

SL_featuresets = [(SL_features(d, SL), c) for (d,c) in phrasedocs_2]

train_set_SL, test_set_SL = train_test_split(SL_featuresets, test_size=0.2, random_state=42)   #42

classifier = nltk.NaiveBayesClassifier.train(train_set_SL)
print(nltk.classify.accuracy(classifier, test_set_SL))

## Bi Gram

In [67]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [68]:
finder = BigramCollocationFinder.from_words(all_words_list_wo_stopwords)

In [69]:
bigram_features = finder.nbest(bigram_measures.chi_sq, 500)
print(bigram_features[:50])

[("'30s", "'40s"), ('10-course', 'banquet'), ('18-year-old', 'mistress'), ('1915', 'armenia'), ('1959', 'godzilla'), ('1962', 'garnish'), ('20,000', 'leagues'), ('20-car', 'pileup'), ('20th-century', 'footnotes'), ('22-year-old', 'girlfriend'), ('9\\/11', 'juxtaposition'), ('\\', '*'), ('abbott', 'ernest'), ('accomplishment', 'veracity'), ('ace', 'japanimator'), ('achieved', 'lottery'), ('action-and-popcorn', 'obsessed'), ('action\\/thriller', 'well-balanced'), ('actorish', 'notations'), ('actory', 'concoctions'), ('adopts', 'guise'), ('adorns', 'family-film'), ('adrien', 'brody'), ('advanced', 'prozac'), ('affirms', 'nourishing'), ('after-hours', 'loopiness'), ('agency', 'boss'), ('agnostic', 'carnivore'), ('aisles', 'bathroom'), ('alert', 'street-smart'), ('all-night', 'tequila'), ('all-star', 'reunions'), ('ally', 'mcbeal-style'), ('american-russian', 'armageddon'), ('amidst', 'swirl'), ('ancillary', 'products'), ('angela', 'gheorghiu'), ('animations', 'constricted'), ('anna', 'moug

In [70]:
def bigram_document_features(document, word_features, bigram_features):
    document_words = set(document)
    document_bigrams = nltk.bigrams(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    for bigram in bigram_features:
        features['B_{}_{}'.format(bigram[0], bigram[1])] = (bigram in document_bigrams)    
    return features

In [ ]:
bigram_featuresets = [(bigram_document_features(d, new_word_features, bigram_features), c) for (d, c) in phrasedocs_2]


## BIgram 0.57575 with stop words remove


In [82]:
train_set_bi, test_set_bi = train_test_split(bigram_featuresets, test_size=0.2, random_state=42)   #42
classifier = nltk.NaiveBayesClassifier.train(train_set_bi)
nltk.classify.accuracy(classifier, test_set_bi)

0.57575

## POS TAG FIRST then BiGram

In [78]:
from nltk import pos_tag

In [79]:
pos_tags_all_words_list = pos_tag(all_words_list_wo_stopwords)


In [80]:
finder_POS = BigramCollocationFinder.from_words(pos_tags_all_words_list)

In [81]:
bigram_features_POS = finder_POS.nbest(bigram_measures.chi_sq, 500)
print(bigram_features_POS[:50])

[(("'30s", 'POS'), ("'40s", 'CD')), (("'50s", 'CC'), ('sociology', 'NN')), (("'em", 'POS'), ('powerment', 'NN')), (('-lrb-', 'FW'), ('goldbacher', 'FW')), (('-rrb-', 'CD'), ('civilized', 'JJ')), (('-rrb-', 'RP'), ('later', 'JJ')), (('10-course', 'JJ'), ('banquet', 'NN')), (('102-minute', 'JJ'), ('infomercial', 'NN')), (('18-year-old', 'JJ'), ('mistress', 'NN')), (('1915', 'CD'), ('armenia', 'NN')), (('1959', 'CD'), ('godzilla', 'NN')), (('1962', 'CD'), ('garnish', 'NN')), (('20,000', 'CD'), ('leagues', 'NNS')), (('20-car', 'JJ'), ('pileup', 'NN')), (('20th-century', 'JJ'), ('footnotes', 'NNS')), (('22-year-old', 'JJ'), ('girlfriend', 'NN')), (('85-minute', 'CD'), ('screwball', 'NN')), (('9\\/11', 'CD'), ('juxtaposition', 'NN')), (('\\', 'CC'), ('*', 'JJ')), (('abbott', 'NN'), ('ernest', 'JJS')), (('abc', 'VBP'), ('africa', 'JJ')), (('abysmal', 'JJ'), ('hannibal', 'NN')), (('acceptable', 'VB'), ('printed', 'JJ')), (('accomplishment', 'JJ'), ('veracity', 'NN')), (('accumulate', 'NN'), ('

In [82]:
bigram_featuresets_POS = [(bigram_document_features(d, new_word_features, bigram_features_POS), c) for (d, c) in phrasedocs]


## 0.56225

In [83]:
train_set_bi_POS, test_set_bi_POS = train_test_split(bigram_featuresets_POS, test_size=0.2, random_state=42)   #42
classifier = nltk.NaiveBayesClassifier.train(train_set_bi_POS)
nltk.classify.accuracy(classifier, test_set_bi_POS)

0.56225

## POS TAG FEATURES WITH Bi-grams

In [55]:
def POS_features(document, word_features):
    document_words = set(document)
    tagged_words = nltk.pos_tag(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    numNoun = 0
    numVerb = 0
    numAdj = 0
    numAdverb = 0
    for (word, tag) in tagged_words:
        if tag.startswith('N'): numNoun += 1
        if tag.startswith('V'): numVerb += 1
        if tag.startswith('J'): numAdj += 1
        if tag.startswith('R'): numAdverb += 1
    features['nouns'] = numNoun
    features['verbs'] = numVerb
    features['adjectives'] = numAdj
    features['adverbs'] = numAdverb
    return features


## POS after only stop words remove

In [61]:
POS_featuresets = [(POS_features(d, new_word_features), c) for (d, c) in phrasedocs_2]


## 0.5533

In [62]:
train_set_POS, test_set_POS = train_test_split(POS_featuresets, test_size=0.2, random_state=42)   #42
classifier = nltk.NaiveBayesClassifier.train(train_set_POS)
nltk.classify.accuracy(classifier, test_set_POS)

0.55325

## LIWC

In [21]:
import os
import sys

# returns two lists:  words in positive emotion class and
#		      words in negative emotion class
def read_words():
  poslist = []
  neglist = []

  flexicon = open('SentimentLexicons/liwcdic2007.dic', encoding='latin1')
  # read all LIWC words from file
  wordlines = [line.strip() for line in flexicon]
  # each line has a word or a stem followed by * and numbers of the word classes it is in
  # word class 126 is positive emotion and 127 is negative emotion
  for line in wordlines:
    if not line == '':
      items = line.split()
      word = items[0]
      classes = items[1:]
      for c in classes:
        if c == '126':
          poslist.append( word )
        if c == '127':
          neglist.append( word )
  return (poslist, neglist)


In [22]:
read_words()

(['accept',
  'accepta*',
  'accepted',
  'accepting',
  'accepts',
  'active*',
  'admir*',
  'ador*',
  'advantag*',
  'adventur*',
  'affection*',
  'agree',
  'agreeab*',
  'agreed',
  'agreeing',
  'agreement*',
  'agrees',
  'alright*',
  'amaz*',
  'amor*',
  'amus*',
  'aok',
  'appreciat*',
  'assur*',
  'attachment*',
  'attract*',
  'award*',
  'awesome',
  'beaut*',
  'beloved',
  'benefic*',
  'benefit',
  'benefits',
  'benefitt*',
  'benevolen*',
  'benign*',
  'best',
  'better',
  'bless*',
  'bold*',
  'bonus*',
  'brave*',
  'bright*',
  'brillian*',
  'calm*',
  'care',
  'cared',
  'carefree',
  'careful*',
  'cares',
  'caring',
  'casual',
  'casually',
  'certain*',
  'challeng*',
  'champ*',
  'charit*',
  'charm*',
  'cheer*',
  'cherish*',
  'chuckl*',
  'clever*',
  'comed*',
  'comfort*',
  'commitment*',
  'compassion*',
  'compliment*',
  'confidence',
  'confident',
  'confidently',
  'considerate',
  'contented*',
  'contentment',
  'convinc*',
  'cool'

In [23]:
(poslist, neglist) = read_words()


In [24]:
def isPresent(word, emotionlist):
  isFound = False
  # loop over all elements of list
  for emotionword in emotionlist:
    # test if a word or a stem
    if not emotionword[-1] == '*':
      # it's a word!
      # when a match is found, can quit the loop with True
      if word == emotionword:
        isFound = True
        break
    else:
      # it's a stem!
      # when a match is found, can quit the loop with True
      if word.startswith(emotionword[0:-1]):
        isFound = True
        break
  # end of loop
  return isFound


In [29]:
def LIWC_features(document, poslist, neglist):
    features = {}
    positive_count = sum(1 for word in document if isPresent(word, poslist))
    negative_count = sum(1 for word in document if isPresent(word, neglist))

    features["positive_count"] = positive_count
    features["negative_count"] = negative_count
    features["sentiment_strength"] = positive_count - negative_count  
    return features

In [30]:
LIWC_featuresets = [(LIWC_features(d, poslist, neglist), c) for (d, c) in phrasedocs_2]


NameError: name 'phrasedocs_2' is not defined

## 0.534

In [83]:
train_set_LIWC, test_set_LIWC = train_test_split(LIWC_featuresets, test_size=0.2, random_state=42)   #42
classifier = nltk.NaiveBayesClassifier.train(train_set_LIWC)
nltk.classify.accuracy(classifier, test_set_LIWC)

0.53

## Combine bigram and LIWC

In [84]:
def combined_features(document, word_features, bigram_features, poslist, neglist):
    
    bigram_feats = bigram_document_features(document, word_features, bigram_features)
    
    
    liwc_features = LIWC_features(document, poslist, neglist)
    
    
    features = {}
    features.update(bigram_feats)  
    features.update(liwc_features)  
    
    return features


In [86]:
combined_featuresets = [
    (combined_features(d, new_word_features, bigram_features, poslist, neglist), c)
    for (d, c) in phrasedocs_2
]

## 0.56325 combined with Bigram features

In [87]:
train_set_combine, test_set_combine = train_test_split(combined_featuresets, test_size=0.2, random_state=42)   #42
classifier = nltk.NaiveBayesClassifier.train(train_set_combine)

nltk.classify.accuracy(classifier, test_set_combine)

0.56325

In [88]:
cross_validation_PRF(num_folds, combined_featuresets, labels)

Each fold size: 6666


KeyboardInterrupt: 

# test

In [14]:
test['Phrase'] = test['Phrase'].fillna('')


In [ ]:
import re
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords





In [ ]:
stemmer = SnowballStemmer('english')
english_stopwords = stopwords.words('english')

In [ ]:
train['Phrase'] = [re.sub(r'\d+', 'num', re.sub(r"[^\s\w]", "", t)).lower() for t in train.Phrase]
test['Phrase'] =  [re.sub(r'\d+', 'num', re.sub(r"[^\s\w]", "", t)).lower() for t in test.Phrase]

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer =lambda text:[stemmer.stem(token) for token in word_tokenize(text) if token not in newstopwords],
                            ngram_range=(1,2),
                            max_features=2000)

In [17]:
vectorizer.fit(train.Phrase)
vectorizer.get_feature_names_out()[:150]

C:\Users\simba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['abandon', 'abil', 'abl', 'absolut', 'absorb', 'abstract',
       'absurd', 'abus', 'accent', 'accept', 'access', 'accomplish',
       'accur', 'ach', 'achiev', 'across', 'act', 'action', 'action film',
       'action movi', 'action sequenc', 'actor', 'actress', 'actual',
       'ad', 'adam', 'adam sandler', 'adapt', 'add', 'addit', 'adequ',
       'admir', 'admit', 'adolesc', 'adult', 'adventur', 'affair',
       'affect', 'afraid', 'afterschool', 'age', 'ago', 'ahead', 'aim',
       'aimless', 'air', 'alien', 'aliv', 'allen', 'allow', 'almost',
       'alon', 'along', 'alreadi', 'also', 'altern', 'although', 'alway',
       'amateurish', 'amaz', 'ambigu', 'ambit', 'ambiti', 'america',
       'american', 'among', 'amount', 'amus', 'analyz', 'angel', 'angst',
       'anim', 'ann', 'annoy', 'anoth', 'answer', 'anyon', 'anyth',
       'apart', 'appar', 'appeal', 'appear', 'appli', 'appreci',
       'approach', 'appropri', 'area', 'argu', 'argument', 'around',
       'arriv', 'art'

In [18]:
from sklearn.model_selection import train_test_split

X = vectorizer.transform(train.Phrase)
y = train.Sentiment
X_test = vectorizer.transform(test.Phrase)


X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [19]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 369497 stored elements and shape (124848, 2000)>

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred1 = model.predict(X_val)

accuracy_score(y_val, y_pred1)

0.630718954248366

In [74]:
def cross_validation_PRF(num_folds, featuresets, labels):
    subset_size = int(len(featuresets)/num_folds)
    print('Each fold size:', subset_size)
    # for the number of labels - start the totals lists with zeroes
    num_labels = len(labels)
    total_precision_list = [0] * num_labels
    total_recall_list = [0] * num_labels
    total_F1_list = [0] * num_labels

    # iterate over the folds
    for i in range(num_folds):
        test_this_round = featuresets[(i*subset_size):][:subset_size]
        train_this_round = featuresets[:(i*subset_size)] + featuresets[((i+1)*subset_size):]
        # train using train_this_round
        classifier = nltk.NaiveBayesClassifier.train(train_this_round)
        # evaluate against test_this_round to produce the gold and predicted labels
        goldlist = []
        predictedlist = []
        for (features, label) in test_this_round:
            goldlist.append(label)
            predictedlist.append(classifier.classify(features))

        # computes evaluation measures for this fold and
        #   returns list of measures for each label
        print('Fold', i)
        (precision_list, recall_list, F1_list) \
                  = eval_measures(goldlist, predictedlist, labels)
        # take off triple string to print precision, recall and F1 for each fold
        '''
        print('\tPrecision\tRecall\t\tF1')
        # print measures for each label
        for i, lab in enumerate(labels):
            print(lab, '\t', "{:10.3f}".format(precision_list[i]), \
              "{:10.3f}".format(recall_list[i]), "{:10.3f}".format(F1_list[i]))
        '''
        # for each label add to the sums in the total lists
        for i in range(num_labels):
            # for each label, add the 3 measures to the 3 lists of totals
            total_precision_list[i] += precision_list[i]
            total_recall_list[i] += recall_list[i]
            total_F1_list[i] += F1_list[i]

    # find precision, recall and F measure averaged over all rounds for all labels
    # compute averages from the totals lists
    precision_list = [tot/num_folds for tot in total_precision_list]
    recall_list = [tot/num_folds for tot in total_recall_list]
    F1_list = [tot/num_folds for tot in total_F1_list]
    # the evaluation measures in a table with one row per label
    print('\nAverage Precision\tRecall\t\tF1 \tPer Label')
    # print measures for each label
    for i, lab in enumerate(labels):
        print(lab, '\t', "{:10.3f}".format(precision_list[i]), \
          "{:10.3f}".format(recall_list[i]), "{:10.3f}".format(F1_list[i]))
    
    # print macro average over all labels - treats each label equally
    print('\nMacro Average Precision\tRecall\t\tF1 \tOver All Labels')
    print('\t', "{:10.3f}".format(sum(precision_list)/num_labels), \
          "{:10.3f}".format(sum(recall_list)/num_labels), \
          "{:10.3f}".format(sum(F1_list)/num_labels))

    # for micro averaging, weight the scores for each label by the number of items
    #    this is better for labels with imbalance
    # first intialize a dictionary for label counts and then count them
    label_counts = {}
    for lab in labels:
      label_counts[lab] = 0 
    # count the labels
    for (doc, lab) in featuresets:
      label_counts[lab] += 1
    # make weights compared to the number of documents in featuresets
    num_docs = len(featuresets)
    label_weights = [(label_counts[lab] / num_docs) for lab in labels]
    print('\nLabel Counts', label_counts)
    #print('Label weights', label_weights)
    # print macro average over all labels
    print('Micro Average Precision\tRecall\t\tF1 \tOver All Labels')
    precision = sum([a * b for a,b in zip(precision_list, label_weights)])
    recall = sum([a * b for a,b in zip(recall_list, label_weights)])
    F1 = sum([a * b for a,b in zip(F1_list, label_weights)])
    print( '\t', "{:10.3f}".format(precision), \
      "{:10.3f}".format(recall), "{:10.3f}".format(F1))
    


In [75]:
def eval_measures(gold, predicted, labels):
    
    # these lists have values for each label 
    recall_list = []
    precision_list = []
    F1_list = []

    for lab in labels:
        # for each label, compare gold and predicted lists and compute values
        TP = FP = FN = TN = 0
        for i, val in enumerate(gold):
            if val == lab and predicted[i] == lab:  TP += 1
            if val == lab and predicted[i] != lab:  FN += 1
            if val != lab and predicted[i] == lab:  FP += 1
            if val != lab and predicted[i] != lab:  TN += 1
        # use these to compute recall, precision, F1
        # for small numbers, guard against dividing by zero in computing measures
        if (TP == 0) or (FP == 0) or (FN == 0):
          recall_list.append (0)
          precision_list.append (0)
          F1_list.append(0)
        else:
          recall = TP / (TP + FP)
          precision = TP / (TP + FN)
          recall_list.append(recall)
          precision_list.append(precision)
          F1_list.append( 2 * (recall * precision) / (recall + precision))

    # the evaluation measures in a table with one row per label
    return (precision_list, recall_list, F1_list)